In [1]:
import cv2
import numpy as np

# 카메라로 영상 재생
## cv2.VideoCapture(0) → 객체생성
    cv2.VideoCapture(0) 함수를 사용해 VideoCapture 객체를 생성한다. VideoCapture의 인자로는 ★장치 인덱스(어떤 카메라를 사용할 것인지, PC에 웹캠이 2개인 경우 첫번째 웹캠은 0, 두번째 웹캠은 1로 지정할 수 있다.) 또는 비디오 파일 이름을 지정한다.

In [2]:
def showVideo():
    try:
        print("카메라를 구동합니다.")
        cap = cv2.VideoCapture(0)
    except:
        print("카메라 구동 실패")
        return

    cap.set(3, 1280)
    cap.set(4, 720)
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("비디오 읽기 오류")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('video', gray)
        
        k = cv2.waitKey(0) & 0xFF
        if k == 27:
            break
            
    print(cap.get(3))
    print(cap.get(4))
    cap.release()
    cv2.destroyAllWindows()

In [3]:
showVideo()

카메라를 구동합니다.
1280.0
720.0


## VideoCapture객체.set(propId, value)
    먼저 VideoCapture(0)으로 비디오 객체, cap을 생성한다. 그 다음 set 함수로 프레임 크기를 지정할 수 있다. 사실 set 함수로 VideoCapture 객체의 다양한 속성들을 정의할 수 있다. 관련 속성들은 cv::VideoCaptureProperties 에서 참고할 수 있다.
    3. cv2.CAP_PROP_POS_AVI_WIDTH : 비디오 스트림에서 가로 길이 지정
    4. cv2.CAP_PROP_POS_AVI_HEIGHT : 비디오 스트림에서 세로 길이 지정
    
    단 기기 하드웨어나 드라이버, API 백엔드 등에 따라서 동작이 조금씩 달라질 수는 있다. 이와 관련된 속성들은 cv::Additional flags for video I/O API backends에서 참고할 수 있다.
    
    (추가로 필요시 VideoCapture객체.get(propId) 메소드를 통해 해당 비디오와 관련된 다양한 속성값을 확인할 수 있다. propId 인자는 set메소드와 마찬가지로 VideoCaptureProperties값을 사용한다.)

    이렇게 라이브로 들어오는 비디오를 프레임별로 캡쳐하고 이를 화면에 보여주어야 하므로, while 문으로 특정 키를 누를 때까지 계속 루프를 돌리며 확인한다.
    
## VideoCapture객체.read() → retval(bool), image
    재생되는 비디오를 한 프레임씩 읽어들이는 함수로 비디오 프레임을 제대로 읽었다면 retval값은 True, 실패하면 False가 저장이 된다. 필요한 경우 retval 값을 체크하여 비디오 프레임을 제대로 읽었는지 확인할 수 있다. 그렇게 불러온 프레임은 image에 저장된다.
    
    가끔 cap이 제대로 초기화 되지 않았을 수도 있는데, 이럴 경우 에러코드를 반환한다. 이 때, VideoCapture객체.isOpened() 메소드를 사용하여 제대로 초기화되었는지 확인할 수 있다. 만약 isOpened()가 False 값을 반환하면 open() 메소드를 이용해 열어주면 된다.

    이후 cvtColor 함수를 이용해서 해당 "프레임 이미지"를 Grayscale 이미지로 변환시켰다. cvtColor 함수와 색공간 변환에 대한 설명은 차후 이미지 Processing에서 공부할 예정이다. 영상은 프레임을 시간에 따라 연속적으로 화면에 보여주는 것이므로 하나의 프레임은 곧 하나의 이미지이다. 따라서 이미지를 화면에 보여주는 함수인 cv2.imshow() 함수를 그대로 사용한다.
    
## VideoCapture객체.release()
    마지막으로 오픈한 VideoCapture 객체를 release() 메소드를 사용해서 반드시 파일을 닫아주어야 한다. 그리고 생성한 모든 윈도우를 제거한다.

# 비디오 파일 불러오기

In [4]:
def readVideo(video_file):
    try:
        cap2 = cv2.VideoCapture(video_file)
        print("영상을 불러옵니다.")
    except:
        print("영상을 불러오는데 실패했습니다.")
        return
    
    while True:
        ret, frame = cap2.read()
        
        if not ret:
            print("영상 재생에 실패했습니다.")
            break
        
        cv2.imshow('Play Video', frame)
        k = cv2.waitKey(0) & 0xFF
        
        if k == 27:
            break
            
    print(cap2.get(3))
    print(cap2.get(4))
    cap2.release()
    cv2.destroyAllWindows()

In [13]:
readVideo('Macro - 34855.mp4')

영상을 불러옵니다.
1280.0
720.0


# 비디오 녹화하고 저장하기
## cv2.VideoWriter(filename, fourcc, fps, frameSize)
    앞서 영상을 가져오기 위해 VideoCapture로 객체를 생성했던 것처럼 영상을 저장할 때에도 VideoWriter를 이용해 객체를 생성해야 한다. 다만 영상을 저장하기 위해서는 영상을 불러오는 것보다 많은 인자들이 필요하다. 우선 파일을 저장할 때 사용되는 이름(filename), 영상을 만들 때 사용하는 코덱(Codec, fourcc), 초당 저장할 프레임(frame per second, fps), 그리고 영상을 저장할 크기(frameSize)이다.

In [7]:
def writeVideo():
    try:
        print("카메라를 구동합니다.")
        cap = cv2.VideoCapture(0)
    except:
        print("카메라 구동 실패")
        return

    fps = 20.0
    width = int(cap.get(3))
    height = int(cap.get(4))
    fcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
    
    out = cv2.VideoWriter('mycam.mp4', fcc, fps, (width, height))
    print('녹화를 시작합니다')
    
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        if not ret:
            print('비디오 읽기 오류')
            break
        
        cv2.imshow('Video', frame)
        out.write(frame)
        
        k = cv2.waitKey(0) & 0xFF
        if k == 27:
            print('녹화를 종료합니다')
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [48]:
writeVideo()

카메라를 구동합니다.
녹화를 시작합니다
녹화를 종료합니다


In [49]:
readVideo('mycam.mp4')

영상을 불러옵니다.
영상 재생에 실패했습니다.
640.0
480.0


    이 후 카메라를 불러와 영상을 찍기 시작하면 프레임 이미지를 읽어와 write 메소드를 통해 프레임 이미지를 저장한다.
    
## VideoWriter객체.write(image)
    write() 메소드에 인자로 프레임 이미지를 전달하면 객체를 생성할 때 설정한 그대로 파일이 생성되며 그 파일에 프레임 이미지가 저장된다.
    
## VideoWriter객체.release()
    녹화 및 저장이 끝났으면 VideoCapture 객체를 release 메소드를 통해 해제시켜주었던 것처럼 VideoWriter 객체도 마찬가지로 release 메소드를 통해 해제시켜주어야 한다.
    
    
    
    
## cv2.flip(src, flipCode [, dst]) → dst
    참고로 read() 메소드를 읽은 후, frame = cv2.flip(frame, 0) 코드를 적용시키면 이미지가 상하 반전되어 읽어진다. frame = cv2.flip(frame, 1) 코드를 적용시키면 코드가 좌우 반전이 된다.